<a href="https://colab.research.google.com/github/RUMONMD89/Computing-science-with-Fortran/blob/main/ACS13REV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/FORTRAN

In [ ]:
%%writefile ACS13.f03
! _/_/_/ N-Body Simulation Using the Direct Method in Lagrangian _/_/_/
! 2022.08.29-09.05 Written by Y.Hirokawa
!
! Governing Eauation: m*d^2x/dt^2 = -sum_j{(G*m*m_j)*(x - x_j)/|x - x_j|^3}
! https://medium.com/swlh/create-your-own-n-body-simulation-with-python-f417234885e9

program main
  implicit none
  integer, parameter          :: NP=100, MAXSTEP=60000, NFREQ=1000, LEAPFROG=0, IPERIOD=0
  double precision, parameter :: SCALE=1.0d1, DTIME=1.0d-2, SOFTENING=0.1
  integer                     :: istep
  double precision            :: dt, g, pi
  double precision            :: dm(NP), x(NP), y(NP), z(NP)
  double precision            :: vx(NP), vy(NP), vz(NP)
  double precision            :: fx(NP,NP,2), fy(NP,NP,2), fz(NP,NP,2)

  ! Initialization
  write(*,*) "[INFO] Initialze Field..."
  call sub_init(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep, pi, SCALE, DTIME)
  write(*,*) "[INFO] dt =", dt, " [s]"

  ! Boundary Condition
  write(*,*) "[INFO] Periodic Boundary Condition (0:Disabled / 1:Enabled):", IPERIOD

  ! Output Initial States
  write(*,*) "Step =", istep, " Time =", dble(istep)*dt, " [s]"
  call sub_output(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep)

  ! Time Marching
  do istep = 1, MAXSTEP

    if(IPERIOD == 0) then
      ! Calculate the Equation with Free Flow Boundary Condition
      call sub_calc(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep, pi, SCALE, SOFTENING, LEAPFROG)
    else
      ! Calculate the Equation with Periodic Boundary Condition
      call sub_calc_periodic(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep, pi, SCALE, SOFTENING, LEAPFROG)
    endif

    if(mod(istep, NFREQ) == 0  .or.  istep == MAXSTEP) then
      ! Output Files
      write(*,*) "Step =", istep, " Time =", dble(istep)*dt, " [s]"
      call sub_output(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep)
    endif
  enddo

  write(*,*) "[INFO] Successfully Completed. "

contains

  subroutine sub_init(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep, pi, SCALE, DTIME)
    integer, intent(in)             :: NP
    integer, intent(inout)          :: istep
    double precision, intent(in)    :: SCALE, DTIME
    double precision, intent(inout) :: dt, g, pi, fx(NP,NP,2), fy(NP,NP,2), fz(NP,NP,2), dm(NP)
    double precision, intent(inout) :: x(NP), y(NP), z(NP), vx(NP), vy(NP), vz(NP)
    integer, parameter              :: NSKIP=100
    integer                         :: nseed, i
    integer, allocatable            :: seed(:)
    double precision                :: epsmac, ave, sigma, tmp1(NP), tmp2(NP)

    ! machine epsilon
    epsmac = epsilon(epsmac)

    ! Pi
    pi = 4.0d0*atan(1.0)
    write(*,*) "[INFO] Pi=",pi

    ! Constant of Gravity
    g = 6.67430d-11  ! [m^3/(kg・s^2)]

    ! Initialize Puseudo Random Number
    call random_seed(size=nseed)
    allocate(seed(nseed))
    seed(:) = 2
    call random_seed(put=seed)

    ! Initialization and Omit the First Part for Good Randomness
    call random_number(tmp1)
    call random_number(tmp2)

    ! Mass
    ! Box-Muller: -3sigma <= 99.7% of tmp1 <= 3sigma for ave=0.0 and sigma=1.0
    ave   = 1.0d3  ! [kg]
    sigma = 3.0d2  ! [kg^2]
    do i = 1, NP
      dm(i) = sigma*sqrt(-2.0d0*log(tmp1(i)))*cos(2.0d0*pi*tmp2(i)) + ave
    enddo

    ! Velocity in x-axis
    call random_number(tmp1)
    call random_number(tmp2)
    ave   = 0.0d0   ! [m/s]
    sigma = 0.0d0 ! [m^2/s^2]
    do i = 1, NP
      vx(i) = sigma*sqrt(-2.0d0*log(tmp1(i)))*cos(2.0d0*pi*tmp2(i)) + ave
    enddo

    ! Velocity in y-axis
    call random_number(tmp1)
    call random_number(tmp2)
    ave   = 0.0d0  ! [m/s]
    sigma = 0.0d0  ! [m^2/s^2]
    do i = 1, NP
      vy(i) = sigma*sqrt(-2.0d0*log(tmp1(i)))*cos(2.0d0*pi*tmp2(i)) + ave
    enddo

    ! Velocity in z-axis
    call random_number(tmp1)
    call random_number(tmp2)
    ave   = 0.0d0  ! [m/s]
    sigma = 0.0d0  ! [m^2/s^2]
    do i = 1, NP
      vz(i) = sigma*sqrt(-2.0d0*log(tmp1(i)))*cos(2.0d0*pi*tmp2(i)) + ave
    enddo

    ! Position in x-axis (0.0 <= x < 1.0*SCALE)
    call random_number(tmp1)
    x(:)  = SCALE*tmp1(:)

    ! Position in y-axis (0.0 <= y < 1.0*SCALE)
    call random_number(tmp1)
    y(:)  = SCALE*tmp1(:)

    ! Position in z-axis (0.0 <= z < 1.0*SCALE)
    call random_number(tmp1)
    z(:)  = SCALE*tmp1(:)

    ! Force
    fx(:,:,:) = 0.0d0
    fy(:,:,:) = 0.0d0
    fz(:,:,:) = 0.0d0

    ! Discrete Time
    dt = DTIME     ! [s]

    ! Timestep
    istep = 0

    return
  end subroutine sub_init


  subroutine sub_calc(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep, pi, SCALE, SOFTENING, LEAPFROG)
    integer, intent(in)             :: NP, LEAPFROG
    integer, intent(in)             :: istep
    double precision, intent(in)    :: pi, SCALE, SOFTENING
    double precision, intent(inout) :: dt, g, fx(NP,NP,2), fy(NP,NP,2), fz(NP,NP,2), dm(NP)
    double precision, intent(inout) :: x(NP), y(NP), z(NP), vx(NP), vy(NP), vz(NP)
    integer                         :: i, j
    double precision                :: tmpx, tmpy, tmpz, tmpdm, tmp

    ! Calculate Force at the Current Step
    do i = 1, NP-1
      do j = i+1, NP
        tmpdm = g*dm(i)*dm(j)
        tmpx  = x(i) - x(j)
        tmpy  = y(i) - y(j)
        tmpz  = z(i) - z(j)
        tmp   = sqrt(tmpx**2 + tmpy**2 + tmpz**2 + SOFTENING**2)
        tmp   = tmpdm/tmp**3
        fx(i,j,1) = tmp*tmpx
        fy(i,j,1) = tmp*tmpy
        fz(i,j,1) = tmp*tmpz
        fx(j,i,1) = -fx(i,j,1)
        fy(j,i,1) = -fy(i,j,1)
        fz(j,i,1) = -fz(i,j,1)
      enddo
    enddo

    ! Update Position
    do i = 1, NP
      x(i) = x(i) + vx(i) - sum(fx(i,:,1))*dt**2/( 2.0d0*dm(i) )
      y(i) = y(i) + vy(i) - sum(fy(i,:,1))*dt**2/( 2.0d0*dm(i) )
      z(i) = z(i) + vz(i) - sum(fz(i,:,1))*dt**2/( 2.0d0*dm(i) )
    enddo

    if(LEAPFROG == 0) then
      ! Euler Method (1st Order)
      ! Update Velocity
      do i = 1, NP
        vx(i) = vx(i) - sum(fx(i,:,1))*dt/( 2.0d0*dm(i) )
        vy(i) = vy(i) - sum(fy(i,:,1))*dt/( 2.0d0*dm(i) )
        vz(i) = vz(i) - sum(fz(i,:,1))*dt/( 2.0d0*dm(i) )
      enddo
    else
      ! Verlet Leap Frog Method (2nd Order)
      ! Calculate Force at the Next Step
      do i = 1, NP-1
        do j = i+1, NP
          tmpdm = g*dm(i)*dm(j)
          tmpx  = x(i) - x(j)
          tmpy  = y(i) - y(j)
          tmpz  = z(i) - z(j)
          tmp   = sqrt(tmpx**2 + tmpy**2 + tmpz**2 + SOFTENING**2)
          tmp   = tmpdm/tmp**3
          fx(i,j,2) = tmp*tmpx
          fy(i,j,2) = tmp*tmpy
          fz(i,j,2) = tmp*tmpz
          fx(j,i,2) = -fx(i,j,2)
          fy(j,i,2) = -fy(i,j,2)
          fz(j,i,2) = -fz(i,j,2)
        enddo
      enddo

      ! Update Velocity
      do i = 1, NP
        vx(i) = vx(i) - sum(fx(i,:,:))*dt/( 2.0d0*dm(i) )
        vy(i) = vy(i) - sum(fy(i,:,:))*dt/( 2.0d0*dm(i) )
        vz(i) = vz(i) - sum(fz(i,:,:))*dt/( 2.0d0*dm(i) )
      enddo
    endif

    return
  end subroutine sub_calc


  subroutine sub_calc_periodic(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep, pi, SCALE, SOFTENING, LEAPFROG)
    integer, intent(in)             :: NP, LEAPFROG
    integer, intent(in)             :: istep
    double precision, intent(in)    :: pi, SCALE, SOFTENING
    double precision, intent(inout) :: dt, g, fx(NP,NP,2), fy(NP,NP,2), fz(NP,NP,2), dm(NP)
    double precision, intent(inout) :: x(NP), y(NP), z(NP), vx(NP), vy(NP), vz(NP)
    ! Number of Ghost Cells
    integer                         :: NGHOST=2
    integer                         :: i, j, m, n, lx, ly, lz, nmax, ldiff
    double precision                :: tmpx, tmpy, tmpz, tmpdm, tmp, fxtmp, fytmp, fztmp
    double precision                :: xl, yl, zl, dl, erfc, fe, htmp, hx, hy, hz, alpha, r, dlnorm

    ! Periodic Boundary Condition (Ver.0 Alpha)

    ! Length of Regular Cube (0 <= x,y,z < 1.0d0*SCALE)
    dl    = 1.0d0*SCALE
    alpha = 2.0d0/dl

    ! Calculate Force at the Current Step
    nmax = 2*NGHOST + 1
    do i = 1, NP-1
      do j = i+1, NP
        tmpdm = g*dm(i)*dm(j)
        fxtmp = 0.0d0
        fytmp = 0.0d0
        fztmp = 0.0d0
        hx    = 0.0d0
        hy    = 0.0d0
        hz    = 0.0d0
        ! Calculate Forces of Ghost Cells [Note: Expand This Loop for Speed-up]
        do n = 0, nmax**3-1
          ! Calculate Shift Length of Ghost Cells
          !
          !  G  G     G  G
          ! +--+--+--+--+--+
          ! |. |. |. |. |. |
          ! +--+--+--+--+--+
          !        ^  ^
          !        |__|
          !         +dl
          !
          lx = mod(n, NMAX)              - NGHOST
          ly = mod(n/NMAX, NMAX)         - NGHOST
          lz = mod(n/(NMAX**2), NMAX**2) - NGHOST
          ldiff  = abs(lx) + abs(ly) + abs(lz)
          dlnorm = sqrt( dble(lx)**2 + dble(ly)**2 + dble(lz)**2 )
          xl = dble(lx)*dl
          yl = dble(ly)*dl
          zl = dble(lz)*dl
          tmpx  = x(i) - x(j) + xl
          tmpy  = y(i) - y(j) + yl
          tmpz  = z(i) - z(j) + zl
          tmp   = sqrt(tmpx**2 + tmpy**2 + tmpz**2 + SOFTENING**2)
          !  === Real Part ===
          r     = alpha*tmp
          erfc  = 1.0d0 - (2.0d0/sqrt(pi))*(r - r**3/3.0d0 + r**5/1.0d1 - r**7/4.2d1 + r**9/2.16d2)
          fe    = 2.0d0*alpha*exp(-alpha**2 * tmp**2)/sqrt(pi)
          fxtmp = fxtmp + tmpx*(erfc + fe)/tmp**3
          fytmp = fytmp + tmpy*(erfc + fe)/tmp**3
          fztmp = fztmp + tmpz*(erfc + fe)/tmp**3
          !
          ! === Imaginary Part ===
          if(ldiff /= 0) then
            htmp = (1.0d0/dlnorm**2)*exp( -(pi**2)*(dlnorm**2)/(alpha**2)*(dl**2) ) &
                   *sin(2.0d0*pi*(hx*tmpx + hy*tmpy + hz*tmpz)/dl)
            hx = hx + hx*htmp
            hy = hy + hy*htmp
            hz = hz + hz*htmp
          endif
        enddo

        fx(i,j,1) = tmpdm*( fxtmp + 2.0d0*hx/dl**2 )
        fy(i,j,1) = tmpdm*( fytmp + 2.0d0*hy/dl**2 )
        fz(i,j,1) = tmpdm*( fztmp + 2.0d0*hz/dl**2 )
        fx(j,i,1) = -fx(i,j,1)
        fy(j,i,1) = -fy(i,j,1)
        fz(j,i,1) = -fz(i,j,1)
      enddo
    enddo

    ! Update Position
    do i = 1, NP
      x(i) = x(i) + vx(i) - sum(fx(i,:,1))*dt**2/( 2.0d0*dm(i) )
      y(i) = y(i) + vy(i) - sum(fy(i,:,1))*dt**2/( 2.0d0*dm(i) )
      z(i) = z(i) + vz(i) - sum(fz(i,:,1))*dt**2/( 2.0d0*dm(i) )
      ! Periodic Boundary [Note: x,y,z must be 0 or positive]
      x(i) = abs(mod(x(i), dl))
      y(i) = abs(mod(y(i), dl))
      z(i) = abs(mod(z(i), dl))
    enddo

    if(LEAPFROG == 0) then
      ! Euler Method (1st Order)
      ! Update Velocity
      do i = 1, NP
        vx(i) = vx(i) - sum(fx(i,:,1))*dt/( 2.0d0*dm(i) )
        vy(i) = vy(i) - sum(fy(i,:,1))*dt/( 2.0d0*dm(i) )
        vz(i) = vz(i) - sum(fz(i,:,1))*dt/( 2.0d0*dm(i) )
      enddo
    else
      ! Verlet Leap Frog Method (2nd Order)
      ! Calculate Force at the Next Step
      do i = 1, NP-1
        do j = i+1, NP
          tmpdm = g*dm(i)*dm(j)
          fxtmp = 0.0d0
          fytmp = 0.0d0
          fztmp = 0.0d0
          hx    = 0.0d0
          hy    = 0.0d0
          hz    = 0.0d0
          ! Calculate Forces of Ghost Cells [Note: Expand This Loop for Speed-up]
          do n = 0, nmax**3-1
            ! Calculate Shift Length of Ghost Cells
            !
            !  G  G     G  G
            ! +--+--+--+--+--+
            ! |. |. |. |. |. |
            ! +--+--+--+--+--+
            !        ^  ^
            !        |__|
            !         +dl
            !
            lx = mod(n, NMAX)              - NGHOST
            ly = mod(n/NMAX, NMAX)         - NGHOST
            lz = mod(n/(NMAX**2), NMAX**2) - NGHOST
            ldiff  = abs(lx) + abs(ly) + abs(lz)
            dlnorm = sqrt( dble(lx)**2 + dble(ly)**2 + dble(lz)**2 )
            xl = dble(lx)*dl
            yl = dble(ly)*dl
            zl = dble(lz)*dl
            tmpx  = x(i) - x(j) + xl
            tmpy  = y(i) - y(j) + yl
            tmpz  = z(i) - z(j) + zl
            tmp   = sqrt(tmpx**2 + tmpy**2 + tmpz**2 + SOFTENING**2)
            !  === Real Part ===
            r     = alpha*tmp
            erfc  = 1.0d0 - (2.0d0/sqrt(pi))*(r - r**3/3.0d0 + r**5/1.0d1 - r**7/4.2d1 + r**9/2.16d2)
            fe    = 2.0d0*alpha*exp(-alpha**2 * tmp**2)/sqrt(pi)
            fxtmp = fxtmp + tmpx*(erfc + fe)/tmp**3
            fytmp = fytmp + tmpy*(erfc + fe)/tmp**3
            fztmp = fztmp + tmpz*(erfc + fe)/tmp**3
            !
            ! === Imaginary Part ===
            if(ldiff /= 0) then
              htmp = (1.0d0/dlnorm**2)*exp( -(pi**2)*(dlnorm**2)/(alpha**2)*(dl**2) ) &
                     *sin(2.0d0*pi*(hx*tmpx + hy*tmpy + hz*tmpz)/dl)
              hx = hx + hx*htmp
              hy = hy + hy*htmp
              hz = hz + hz*htmp
            endif
          enddo

          fx(i,j,2) = tmpdm*( fxtmp + 2.0d0*hx/dl**2 )
          fy(i,j,2) = tmpdm*( fytmp + 2.0d0*hy/dl**2 )
          fz(i,j,2) = tmpdm*( fztmp + 2.0d0*hz/dl**2 )
          fx(j,i,2) = -fx(i,j,2)
          fy(j,i,2) = -fy(i,j,2)
          fz(j,i,2) = -fz(i,j,2)
        enddo
      enddo

      ! Update Velocity
      do i = 1, NP
        vx(i) = vx(i) - sum(fx(i,:,:))*dt/( 2.0d0*dm(i) )
        vy(i) = vy(i) - sum(fy(i,:,:))*dt/( 2.0d0*dm(i) )
        vz(i) = vz(i) - sum(fz(i,:,:))*dt/( 2.0d0*dm(i) )
      enddo
    endif

    return
  end subroutine sub_calc_periodic


  subroutine sub_output(dt, g, dm, x, y, z, vx, vy, vz, fx, fy, fz, NP, istep)
    integer, intent(in)             :: NP
    integer, intent(in)             :: istep
    double precision, intent(inout) :: dt, g, fx(NP,NP,2), fy(NP,NP,2), fz(NP,NP,2), dm(NP)
    double precision, intent(inout) :: x(NP), y(NP), z(NP), vx(NP), vy(NP), vz(NP)
    double precision                :: time
    integer                         :: i, iu
    character                       :: cfname*15, cstep*8, cprefix*3, cextension*4

    ! Casting integer to character
    write(cstep,'(I8.8)') istep

    ! Prefix
    cprefix = 'pp_'

    ! Extension
    cextension = '.csv'

    ! File Name
    cfname = trim(cprefix) // adjustl(cstep) // cextension

    ! File Open (Reserved Unit Number = 5:Keyboard, 6:Screen)
    iu = 10
    open(unit=iu, file=cfname, form='formatted', status='replace')
    write(iu,*) "time, dm, x, y, z, vx, vy, vz"

    ! Calculate Time (Step starts from 0)
    time = istep*dt

    ! Calculate Position and Write Data (i,j,k starts from 1)
    do i = 1, NP
      write(iu,*) time,', ',dm(i),', ',x(i),', ',y(i),', ',z(i),', ',vx(i),', ',vy(i),', ',vz(i)
    enddo
    close(iu)

    return
  end subroutine sub_output

end program main

In [ ]:
DEBUG=0
EXE="ACS13"

if DEBUG == 0:
  !gfortran {EXE}.f03 -o ./{EXE}.out
  !./{EXE}.out
else:
  !sudo apt install gdb
  !gfortran {EXE}.f03 -o ./{EXE}.out -g
  !echo "run" | gdb ./{EXE}.out